In [ ]:
import os

api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

llm_model = 'gpt-3.5-turbo'


In [2]:
from typing import List
from pydantic import BaseModel, Field

In [3]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [ ]:
foo = User(name="Joe",age=32, email="joe@gmail.com")
foo.name

In [5]:
foo = User(name="Joe",age="bar", email="joe@gmail.com")

In [ ]:
foo.age

In [7]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [8]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [ ]:
foo_p

In [ ]:
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

In [12]:
class Class(BaseModel):
    students: List[pUser]

In [13]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [ ]:
obj

In [15]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [16]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [ ]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

In [ ]:
weather_function

In [19]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

In [ ]:
convert_pydantic_to_openai_function(WeatherSearch1)

In [21]:
from langchain.chat_models import ChatOpenAI

In [ ]:
model = ChatOpenAI()

In [ ]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

In [26]:
model_with_function = model.bind(functions=[weather_function])

In [ ]:
model_with_function

In [ ]:
model_with_function.invoke("what is the weather in SF today?")

In [29]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [ ]:
model_with_forced_function.invoke("HI!")

Using in a chain

In [33]:
from langchain.prompts import ChatPromptTemplate

In [34]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [35]:
chain = prompt | model_with_function

In [ ]:
chain.invoke({"input": "what is the weather in sf?"})

In [37]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [38]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

In [39]:
model_with_functions = model.bind(functions=functions)

In [ ]:
model_with_functions.invoke("what is the weather in sf?")

In [ ]:
model_with_functions.invoke("what are three songs by taylor swift?")

In [ ]:
model_with_functions.invoke("hi!")